In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

import zipfile
import pickle
from os.path import join as path, dirname
try:
    from IPython.core.display import HTML

    def pprint(df):
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(HTML(pd.DataFrame(df).to_html()))
except:
    def pprint(df):
        print(df)

In [23]:
print('Training data:')
df_train = pd.read_csv(path('data', 'train.csv'), index_col='PassengerId')
print(df_train.shape)
pprint(df_train.head())


print('Test data:')
df_test = pd.read_csv(path('data', 'test.csv'), index_col='PassengerId')
print(df_test.shape)
pprint(df_test.head())

Training data:
(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Test data:
(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [111]:
def normalize_df(df):
    df.loc[(df["Sex"]=="male"), "Sex"]=0
    df.loc[(df["Sex"]=="female"), "Sex"]=10
    
    df.loc[(np.isnan(df["Age"])), "Age"]=df["Age"].mean()
    df['Age'] = (df['Age']-0)/(100-0)
    
    df.loc[(np.isnan(df["Fare"])), "Fare"]=df["Fare"].mean()
    df['Fare'] = (df['Fare']-0)/(1000-0)
    
    df.loc[df["Cabin"].notnull(), "Cabin"]=1
    df.loc[(df["Cabin"]!=1), "Cabin"]=0
    
    df.loc[(df["Embarked"]=="C"), "Embarked"]=0
    df.loc[(df["Embarked"]=="Q"), "Embarked"]=1
    df.loc[(df["Embarked"]=="S"), "Embarked"]=2

In [112]:
X_full = df_train.copy()
y = X_full.Survived
X_full.drop(["Survived","Name", "Ticket"], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)
normalize_df(X_train_full)
print('Training data:')
print(X_train_full.shape)
pprint(X_train_full.head())

Training data:
(712, 8)


C:\Users\usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-111-9f5bba3a31d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = (df['Age']-0)/(100-0)
<ipython-input-111-9f5bba3a31d8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,
141,3,10,0.297452,0,2,0.015246,0,0
440,2,0,0.310000,0,0,0.010500,0,2
818,2,0,0.310000,1,1,0.037004,0,0
379,3,0,0.200000,0,0,0.004013,0,0
492,3,0,0.210000,0,0,0.007250,0,2


In [113]:
normalize_df(X_valid_full)
print('Validation data:')
print(X_valid_full.shape)
pprint(X_valid_full.head())

Validation data:
(179, 8)


C:\Users\usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-111-9f5bba3a31d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = (df['Age']-0)/(100-0)
<ipython-input-111-9f5bba3a31d8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,
496,3,0,0.295152,0,0,0.014458,0,0
649,3,0,0.295152,0,0,0.007550,0,2
279,3,0,0.070000,4,1,0.029125,0,1
32,1,10,0.295152,1,0,0.146521,1,0
256,3,10,0.290000,0,2,0.015246,0,0


In [114]:
def similarity(x, y):
    """
    Definir quina de les similituds vols utilitzar  a l'execució.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud
    """
    return euclid(x, y)
    
def euclid(x, y):
    """
    Retorna la distància euclidiana inversa de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distància euclidiana
    """
    return 1/(1+np.linalg.norm(x-y, ord=2))

def cosine(x, y):
    """
    Retorna la similitud cosinus de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud cosinus
    """
    x_norm = np.linalg.norm(x, ord=2)
    y_norm = np.linalg.norm(y, ord=2)
    
    # Si alguna de les dues normes es 0, significa que totes les components d'un dels dos vectors son 0, es a dir,
    # que no tenim informacio d'un dels dos vectors. Aixi, retornem -1 com a resultat perque no estan gens relacionats.
    if (x_norm == 0 or y_norm == 0):
        return -1
    #Altrament retornem la formula de la similitud cosinus
    return np.dot(x, y)/(x_norm * y_norm)

def pearson(x, y):
    """
    Retorna la correlació de Pearson de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la correlació de Pearson
    """
    first = x - np.mean(x)
    second = y - np.mean(y)
    norm_first = np.linalg.norm(first, ord=2)
    norm_second = np.linalg.norm(second, ord=2)
    
    # Si alguna de les dues normes es 0, significa que un dels dos vectors te totes les components iguals i a la
    # formula estariem dividint per 0. En aquest cas, hem decidit retornar la similitud cosinus entre els dos vectors
    # perque la similitud cosinus distingira entre si totes les components del vector son 0 o be iguals pero diferent de 0.
    if ((norm_first == 0) or (norm_second == 0)):
        return cosine(x, y)
    #Altrament retornem la formula de la correlacio de Pearson
    return np.dot(first, second)/(norm_first*norm_second)

In [115]:
def find_most_similar(r, df):
    best_index = -1
    best_sim = -1
    for i, r_aux in df.iterrows():
        sim = similarity(r,r_aux) 
        if (sim > best_sim):
            best_sim = sim
            best_index = i
    if (best_index == -1):
        print(r)
    return best_index

In [116]:
def make_preds(xtrain, ytrain, test):
    preds = []
    for index, row in test.iterrows():
        b_index = find_most_similar(row, xtrain)
        preds.append(ytrain.loc[b_index])
    return preds

In [117]:
preds = make_preds(X_train_full, y_train, X_valid_full)
print(preds)

[0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]


In [118]:
def check_preds(predictions, valids):
    i = 0
    encerts = 0
    total = 0
    for row in valids:
        if (row == predictions[i]):
            encerts += 1
        total +=1.
        i += 1
    return encerts/total
check_preds(preds, y_valid)

0.7318435754189944

In [119]:
X_full = df_train.copy()
y = X_full.Survived
X_full.drop(["Survived","Name", "Ticket"], axis=1, inplace=True)

X_test = df_test.copy()
X_test.drop(["Name", "Ticket"], axis=1, inplace=True)

normalize_df(X_full)
normalize_df(X_test)

In [120]:
results = make_preds(X_full, y, X_test)

In [121]:
results = pd.Series(results)
results.index.name = 'PassengerId'
results.index = X_test.index
results.name = 'Survived'
results.to_frame().to_csv('submission.csv')

In [ ]:
best_submission = pd.read_csv("best_submission.csv", index_col='PassengerId')
print(type(best_submission["Survived"]))
for item, value in best_submission["Survived"].iteritems():
    if (item != "Survived"):
        if(value != results.loc[item]):
            print("Diferencia")